In [2]:
import os

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import random
from io import BytesIO

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))

TRAIN_DIR = 'classification/data/train'
VAL_DIR   = 'classification/data/validation'

train_dogs_dir   = os.path.join(TRAIN_DIR, 'dogs')
train_cats_dir   = os.path.join(TRAIN_DIR, 'cats')
validate_dogs_dir = os.path.join(VAL_DIR, 'dogs')
validate_cats_dir = os.path.join(VAL_DIR, 'cats')

In [8]:
print(f"total training dogs images: {len(os.listdir(train_dogs_dir))}")
print(f"total training cats images: {len(os.listdir(train_cats_dir))}")
print(f"total validation dogs images: {len(os.listdir(validate_dogs_dir))}")
print(f"total validation cats images: {len(os.listdir(validate_cats_dir))}")

FileNotFoundError: [Errno 2] No such file or directory: 'classification/data/train/dogs'